# 02 – Forecasting Cash Flow and CVaR Buffer Sizing

Objective:
- Forecast net operating cash flows.
- Compute CVaR-based buffer to protect against shortfall risk.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from value_ops_lab.synth import make_synthetic
from value_ops_lab.risk_models import quantile_forecast, cvar_cash_buffer

# Generate synthetic dataset
df = make_synthetic(n_months=36)
df["net_cf"] = df["sales"] - df["cogs"]

# Features: simple lag features
df["lag1"] = df["net_cf"].shift(1)
df["lag2"] = df["net_cf"].shift(2)
df = df.dropna()

X = df[["lag1","lag2"]]
y = df["net_cf"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Forecast next 6 months
X_future = X_test.tail(6)
preds = quantile_forecast(X_train, y_train, X_future, quantiles=(0.1,0.5,0.9))
preds

In [ ]:
# CVaR buffer sizing
scenarios = y_test.tail(12).values + np.random.normal(0,5000,12)
buffer, t = cvar_cash_buffer(scenarios, alpha=0.95)
print(f"Optimal buffer at 95% CVaR: ${buffer:,.0f}")

### Key Insight
- The CVaR buffer balances cost of capital vs. probability of cash shortfall.
- This aligns with **risk-aware FP&A**, highly relevant to Accordion’s analytics.